# SEC Format

It turns out there’s already a standard for serializing ECDSA public keys called SEC. SEC stands for Standards for Efficient Cryptography and as the name suggests, it’s got minimal overhead.

![SEC Formats](./sec_format.png)

In [1]:
# SEC Example

from ecc import S256Point

point = S256Point(0x914fa759ff2a9ea88748805d7703fbe48dd1ede20394585ef1551a1200b957f8, 0x6AEBCA40BA255960A3178D6D861A54DBA813D0B813FDE7B5A5082628087264DA)

uncompressed = b'\x04' + point.x.num.to_bytes(32, 'big') + point.y.num.to_bytes(32, 'big')
print(uncompressed.hex())
if point.y.num % 2 == 1:
    compressed = b'\x03' + point.x.num.to_bytes(32, 'big')
else:
    compressed = b'\x02' + point.x.num.to_bytes(32, 'big')
print(compressed.hex())

RuntimeError: (914fa759ff2a9ea88748805d7703fbe48dd1ede20394585ef1551a1200b957f8, 6aebca40ba255960a3178d6d861a54dba813d0b813fde7b5a5082628087264da) is not on the curve

### Try it

#### 6.1. Find the compressed and uncompressed SEC format for pub keys where the private keys are:
```
999**3, 123, 42424242
```

In [4]:
from ecc import G

secrets = (999**3, 123, 42424242)

# iterate through secrets
for secret in secrets:
    # get public point
    point = secret * G
    # uncompressed - b'\x04' followed by x coord, then y coord
    # here's how you express a coordinate in bytes: some_integer.to_bytes(32, 'big')
    uncompressed = b'\x04' + point.x.num.to_bytes(32, 'big') + point.y.num.to_bytes(32, 'big')
    # compressed - b'\x02'/b'\x03' follewed by x coord. 02 if y is even, 03 otherwise
    if point.y.num % 2 == 1:
        compressed = b'\x03' + point.x.num.to_bytes(32, 'big')
    else:
        compressed = b'\x02' + point.x.num.to_bytes(32, 'big')
    # print the .hex() of both
    print(uncompressed.hex())
    print(compressed.hex())

049d5ca49670cbe4c3bfa84c96a8c87df086c6ea6a24ba6b809c9de234496808d56fa15cc7f3d38cda98dee2419f415b7513dde1301f8643cd9245aea7f3f911f9
039d5ca49670cbe4c3bfa84c96a8c87df086c6ea6a24ba6b809c9de234496808d5
04a598a8030da6d86c6bc7f2f5144ea549d28211ea58faa70ebf4c1e665c1fe9b5204b5d6f84822c307e4b4a7140737aec23fc63b65b35f86a10026dbd2d864e6b
03a598a8030da6d86c6bc7f2f5144ea549d28211ea58faa70ebf4c1e665c1fe9b5
04aee2e7d843f7430097859e2bc603abcc3274ff8169c1a469fee0f20614066f8e21ec53f40efac47ac1c5211b2123527e0e9b57ede790c4da1e72c91fb7da54a3
03aee2e7d843f7430097859e2bc603abcc3274ff8169c1a469fee0f20614066f8e


### Test Driven Exercise

In [ ]:
from ecc import S256Point
from helper import double_sha256, encode_base58, hash160

class S256Point(S256Point):

    def sec(self, compressed=True):
        # returns the binary version of the sec format, NOT hex
        # if compressed, starts with b'\x02' if self.y.num is even, b'\x03' if self.y is odd
        # then self.x.num
        if compressed:
            if self.y.num % 2 == 0:
                return b'\x02' + self.x.num.to_bytes(32, 'big')
            else:
                return b'\x03' + self.x.num.to_bytes(32, 'big')
        # remember, you have to convert self.x.num/self.y.num to binary (some_integer.to_bytes(32, 'big'))
        # if non-compressed, starts with b'\x04' followod by self.x and then self.y
        else:
            return b'\x04' + self.x.num.to_bytes(32, 'big') + self.y.num.to_bytes(32, 'big')
        
        
        